In [1]:
## 설치
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [773 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,356 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubun

In [2]:
## 초기화
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [3]:
import pandas as pd
import pyspark
import pyspark.sql.functions as f
from pyspark.sql import DataFrame, SparkSession, Window
from pyspark.sql.functions import spark_partition_id
from pyspark.sql.types import StructType, StructField, IntegerType, ShortType, StringType

In [4]:
## Session 생성
spark = (
        SparkSession
        .builder
        .appName("Commerce Session")
        .getOrCreate())

spark

In [5]:
def get_spark_session():
    ## Session 생성
    spark = (
        SparkSession
        .builder
        .appName("Commerce Session")
        .getOrCreate()
    )

    return spark

In [6]:
spark

- inferSchema를 하면 스파크 보고 알아서 스키마를 하라는 것
- order_id는 integertype을 쓰라.. 이렇게 구체적으로 알려주기
- header도 꼭 True로 바꿔쓰기

In [7]:
%%time
df = spark.read.csv("/content/order_products__prior.csv", header=True, inferSchema=True)
df.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


In [8]:
struct_schema = StructType([
    StructField("order_id", IntegerType()),
    StructField("product_id", IntegerType()),
    StructField("add_to_cart_order", ShortType()),
    StructField("reordered", ShortType())
    ])

df = spark.read.csv('/content/order_products__prior.csv',header=True,schema = struct_schema)
df.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       2|     33120|                1|        1|
|       2|     28985|                2|        1|
|       2|      9327|                3|        0|
|       2|     45918|                4|        1|
|       2|     30035|                5|        0|
|       2|     17794|                6|        1|
|       2|     40141|                7|        1|
|       2|      1819|                8|        1|
|       2|     43668|                9|        0|
|       3|     33754|                1|        1|
|       3|     24838|                2|        1|
|       3|     17704|                3|        1|
|       3|     21903|                4|        1|
|       3|     17668|                5|        1|
|       3|     46667|                6|        1|
|       3|     17461|                7|        1|
|       3|     32665|                8|        1|


In [9]:
df1 = spark.read.csv("/content/order_products__prior.csv", header=True, schema=struct_schema)
df2 = spark.read.csv("/content/order_products__train.csv", header=True, schema=struct_schema)

df = df1.union(df2)
# pandas의 콘캣과 유사하다.
df = df.filter("order_id % 13 = 1")

In [13]:
import pyspark
from pyspark.sql import DataFrame, SparkSession

In [14]:
from pyspark.sql.types import StringType

struct_schema_product = StructType([
    StructField("product_id", IntegerType()),
    StructField("product_name", StringType()),
    StructField("aisle_id", IntegerType()),
    StructField("department_id", IntegerType())
])
struct_schema_aisle = StructType([
    StructField("aisle_id", IntegerType()),
    StructField("aisle", StringType())
])
struct_schema_depart = StructType([
    StructField("department_id", IntegerType()),
    StructField("department", StringType())
])


df_product = (
    spark.read.csv("/content/products.csv", header=True, schema=struct_schema_product)
    .join(spark.read.csv("/content/aisles.csv", header=True, schema=struct_schema_aisle), on="aisle_id", how="left")
    .join(spark.read.csv("/content/departments.csv", header=True, schema=struct_schema_depart), on="department_id", how="left")
    .drop("aisle_id", "department_id")
)

In [15]:
df = df.join(df_product, on="product_id", how="left")

In [16]:
df.persist()

DataFrame[product_id: int, order_id: int, add_to_cart_order: smallint, reordered: smallint, product_name: string, aisle: string, department: string]

In [17]:
%%time
df.show()

+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|               aisle|  department|
+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|     41890|      14|                6|        1|Organic Mesa Sunr...|              cereal|   breakfast|
|     26706|     976|               11|        1|Chicken Pad Thai ...|        frozen meals|      frozen|
|     19204|    1483|                5|        1|Organic Heritage ...|              cereal|   breakfast|
|     23015|    1561|                7|        1|Breaded Chicken N...|frozen appetizers...|      frozen|
|     15790|    1587|                3|        1|Organic Veggie Bites|frozen vegan vege...|      frozen|
|     28836|    2003|               15|        0|    Grapefruit Juice|        refrigerated|   beverages|
|     28836|    2458|               21|        1|    Gr

In [18]:
df.rdd.getNumPartitions()

200

In [19]:
from pyspark.sql.functions import spark_partition_id

df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 1866|
|          1| 2554|
|          2| 2221|
|          3| 9037|
|          4| 3181|
|          5| 2069|
|          6| 2714|
|          7| 1182|
|          8| 3346|
|          9| 2381|
|         10| 3229|
|         11| 3455|
|         12| 2783|
|         13| 8025|
|         14| 2449|
|         15| 2104|
|         16| 4237|
|         17| 2499|
|         18| 1736|
|         19| 2788|
+-----------+-----+
only showing top 20 rows



In [20]:
df = df.repartition("order_id")
df.persist()
df.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count().show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 3119|
|          1| 3370|
|          2| 3201|
|          3| 3269|
|          4| 3070|
|          5| 3257|
|          6| 3316|
|          7| 2957|
|          8| 2839|
|          9| 3008|
|         10| 3129|
|         11| 3018|
|         12| 3268|
|         13| 3362|
|         14| 3103|
|         15| 3367|
|         16| 3573|
|         17| 3384|
|         18| 2852|
|         19| 3479|
+-----------+-----+
only showing top 20 rows



In [23]:
%%time

import pyspark.sql.functions as f
df_last_order_short = df.orderBy(["order_id", "add_to_cart_order"]).groupBy("order_id").agg(f.last("add_to_cart_order")).toPandas()

CPU times: user 520 ms, sys: 13 ms, total: 533 ms
Wall time: 15.1 s


In [21]:
%%time

import pyspark.sql.functions as f
from pyspark.sql import Window

w = Window.partitionBy('order_id')
df_last_order = df.withColumn('max_add_to_cart_order', f.max('add_to_cart_order').over(w))\
    .where(f.col('add_to_cart_order') == f.col('max_add_to_cart_order'))\
    .drop('max_add_to_cart_order')
df_last_order_long = df_last_order.toPandas()

CPU times: user 559 ms, sys: 66 ms, total: 625 ms
Wall time: 9.19 s


In [24]:
df_temp = df_last_order_long.merge(df_last_order_short, on="order_id")

In [25]:
%%time
df_last_order.show()

+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|               aisle|  department|
+----------+--------+-----------------+---------+--------------------+--------------------+------------+
|      1991|     833|                6|        1|Organic Lemonade ...|       juice nectars|   beverages|
|     22474|    6397|                7|        0|Cheddar Bunnies S...|            crackers|      snacks|
|     38274|    7554|               10|        0|Ice Cream Sandwic...|       ice cream ice|      frozen|
|     26348|    9465|                2|        0|Mixed Fruit Fruit...|fruit vegetable s...|      snacks|
|     30169|   10206|                4|        1|Total 2% All Natu...|              yogurt|  dairy eggs|
|     25194|   10362|               36|        1|Ground Sausage St...|tofu meat alterna...|        deli|
|     47209|   10817|                7|        0|Organi

In [26]:
%%time
import pandas as pd

department_count = pd.concat([
    df_last_order.groupby("department").count().toPandas(),
    df.groupby("department").count().toPandas()
], axis=1)

CPU times: user 116 ms, sys: 19.5 ms, total: 136 ms
Wall time: 16 s


In [27]:
department_count

,department,count,department,count
0,beverages,6187,meat seafood,13658
1,meat seafood,1348,beverages,52749
2,frozen,4676,frozen,44465
3,deli,2099,deli,20435
4,dry goods pasta,1583,dry goods pasta,17162
5,other,106,bulk,659
6,babies,527,other,718
7,bakery,2166,babies,8109
8,produce,15919,bakery,22799
9,pantry,4321,produce,184964


In [28]:
%%time
df = df.orderBy(["order_id", "add_to_cart_order"])

df_temp = df.select(*df.columns, f.lag("aisle").over(w.partitionBy(f.lit(1)).orderBy("order_id")).alias("aisle_before"))
df_temp = df_temp.filter(f.col("aisle") != f.col("aisle_before"))
df_temp = df_temp.filter(f.col("add_to_cart_order") != 1).toPandas()

CPU times: user 4.52 s, sys: 644 ms, total: 5.17 s
Wall time: 28.5 s
